# Trends Plotting

This notebook is intended to showcase `dqmexplore`'s trend plotting tools.

## Setup

If you are working in SWAN, you will need to install the `omsapi` as well as `dqmexplore` itself.

In [ ]:
# Run this if you are in SWAN
# Package installation/updating
!pip3 install -e .. --no-dependencies
!pip3 install omsapi
!pip3 install cmsdials --upgrade

### Imports

In [ ]:
# DIALS API
# For more information on DIALS, please visit https://github.com/cms-DQM/dials-py
import cmsdials
from cmsdials.auth.client import AuthClient
from cmsdials.auth.bearer import Credentials
from cmsdials import Dials
from cmsdials.filters import LumisectionHistogram1DFilters, LumisectionHistogram2DFilters

auth = AuthClient()
token = auth.device_auth_flow()
creds = Credentials.from_authclient_token(token)
dials = Dials(creds)

Run your prefered authentication method for the OMS API. For more information on the OMS API, please visit https://gitlab.cern.ch/cmsoms/oms-api-client

In [ ]:
# Method 1: krb
import omsapi

oms_fetch = omsapi.OMSAPI("https://cmsoms.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_krb()

In [ ]:
# Method 2: oidc
import omsapi
import json

with open("./clientid.json", "r") as file:
    secrets = json.load(file)

oms_fetch = omsapi.OMSAPI("http://vocms0185.cern.ch/agg/api", "v1", cert_verify=False)
oms_fetch.auth_oidc(secrets["API_CLIENT_ID"], secrets["API_CLIENT_SECRET"], audience="cmsoms-int-0185")

del(secrets)
del(file)

In [ ]:
# DQMExplore
import dqmexplore
from dqmexplore.dataproc import generate_me_dict
from dqmexplore.omsutils import get_rate, plot_rate
from dqmexplore.trends import compute_trends, plot_trends 

# Plotly
import plotly
import plotly.graph_objects as go
from plotly.offline import plot

# Other
import pandas as pd
import numpy as np

If you run the following, you will get a list of all the available 1D MEs.

In [ ]:
mes_df = pd.DataFrame([me_qry_rslt.__dict__ for me_qry_rslt in dials.mes.list()])
mes_df[mes_df["dim"] == 1]

## Using OMS to Obtain Metadata

Using the OMS API, we can access important information regarding the run conditions and other information about the run. The available endpoints are:

* `lumisections`
* `runs`
* `fills`
* `datasetrates`

You can access the trigger rate* in the following way:

*HLT ZeroBias trigger rate

In [ ]:
runnb = 380238
omstrig_df = get_rate(oms_fetch, runnb, "ZeroBias")
fig = plot_rate(omstrig_df, norm=False, show=True)

# Optionally, you can export the Plotly figure object dqmexplore returns. Just remember to set show=False in plot_rate
# plot(fig, filename=f"./plots/trigrate_{runnb}.html")

## Trends

In [ ]:
runnb = 380238
me__regex =  "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_(1|2)" 

data1D = dials.h1d.list_all(
    LumisectionHistogram1DFilters(
        run_number = runnb,
        dataset__regex = "ZeroBias",
        me__regex = me__regex
    ),
    # max_pages=200
).to_pandas()
data1D_dict = generate_me_dict(data1D)
omstrig_arr= get_rate(oms_fetch, runnb, "ZeroBias", dataframe=False)

In [ ]:
# Computing trend variables
trends = compute_trends(data1D, trigger_rates = omstrig_arr)

me = "PixelPhase1/Tracks/PXBarrel/charge_PXLayer_1"
trend_names = np.array(["Mean", "Standard Deviation", "Max"])
ylabels = np.char.add(np.array(
    ["Charge (e) "] * 3), 
    trend_names
)
fig_titles = np.char.add(
    [f"Run {runnb} PX Layer 1 Charge "] * 3,
    trend_names
)

# Plotting trend variables
plot_trends(trends, me, fig_titles=fig_titles, norm=False, ylabels=ylabels, log=False)